In [164]:
from util import read_csv

import numpy as np
import pandas as pd

In [165]:
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

In [166]:
df = read_csv('data', 'combined_data_set.csv')

x:\Documents\Programming\dawr\dawr-grisons\src\util.py:45: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.replace('---', pd.np.nan, inplace=True)


In [167]:
df['Bevoelkerungsdichte_normalized_inverse'] = 1 - (df['Bevoelkerungsdichte_pro_km2'] - df['Bevoelkerungsdichte_pro_km2'].min()) / (df['Bevoelkerungsdichte_pro_km2'].max() - df['Bevoelkerungsdichte_pro_km2'].min())
df['Auslaender_in_prozent_normalized_inverse'] = (df['Auslaender_in_prozent'].max() - df['Auslaender_in_prozent']) / (df['Auslaender_in_prozent'].max() - df['Auslaender_in_prozent'].min())
df['Buenzli_Rank_ranking_1'] = ((0.7 * df['Bevoelkerungsdichte_normalized_inverse'] + 1.3 * df['Auslaender_in_prozent_normalized_inverse']) / 2) * 100

df['Bevoelkerungsdichte_normalized']
df['Einwohner_normalized']
df['Twenty_to_Sixtyfour_normalized']

cols_to_show = ['Gemeinde_Name', 'Buenzli_Rank_ranking_1']
df = df.sort_values(by='Buenzli_Rank_ranking_1', ascending=False)
print(df[cols_to_show])

           Gemeinde_Name  Buenzli_Rank_ranking_1
73                 Furna               99.659885
23            Tschappina               99.177956
76  Conters im Prättigau               98.318343
25             Safiental               98.166074
34    Muntogna da Schons               97.907019
..                   ...                     ...
90             Landquart               44.706145
55            Silvaplana               43.486438
22                Thusis               38.233009
79                  Chur               35.358244
52            St. Moritz               26.329352

[101 rows x 2 columns]


In [168]:
famous_count_surses = df[df['Gemeinde_Name'] == 'Surses']['Famous_Person_count'].reset_index(drop=True)
famous_count_vaz = df[df['Gemeinde_Name'] == 'Vaz/Obervaz']['Famous_Person_count'].reset_index(drop=True)

print(famous_count_surses)
print(famous_count_vaz)

print (famous_count_vaz > famous_count_surses)


0    0.0
Name: Famous_Person_count, dtype: float64
0    10.0
Name: Famous_Person_count, dtype: float64
0    True
Name: Famous_Person_count, dtype: bool


In [169]:
# Ranking stuff
# create a sample dataframe
df = pd.DataFrame({'Name': ['Alice', 'Bob', 'Charlie', 'David'],
                   'GPA': [3.5, 3.2, 3.9, 3.6],
                   'Test Scores': [85, 90, 95, 80],
                   'Attendance': [90, 95, 80, 85],
                   'Participation': [75, 80, 85, 90]})
                   
# normalize the factors using min-max normalization
normalized = (df[['GPA', 'Test Scores', 'Attendance', 'Participation']] 
              - df[['GPA', 'Test Scores', 'Attendance', 'Participation']].min()) / (df[['GPA', 'Test Scores', 'Attendance', 'Participation']].max() 
                   - df[['GPA', 'Test Scores', 'Attendance', 'Participation']].min())

# combine the normalized factors into a single score
df['Ranking Score'] = normalized.mean(axis=1) * 100

In [170]:
# define the weights for each factor
weights = {'GPA': 0.3, 'Test Scores': 0.4, 'Attendance': 0.2, 'Participation': 0.1}

# multiply each column by its weight factor
weighted = df[['GPA', 'Test Scores', 'Attendance', 'Participation']].multiply(pd.Series(weights), axis=1)

# normalize the weighted factors using min-max normalization
normalized = (weighted - weighted.min()) / (weighted.max() - weighted.min())

# combine the normalized, weighted factors into a single score
df['Ranking Score'] = normalized.mean(axis=1) * 100

In [171]:
# define the weights for each factor
weights = {'GPA': 0.3, 'Test Scores': 0.4, 'Attendance': 0.2, 'Participation': 0.1}

# define a function to calculate conditional weights
def get_weight(value):
    if np.isnan(value) or value == 1:
        return 0
    elif value in [2, 3, 5, 7]:
        return 2
    else:
        return 1

# calculate the conditional weights for the GPA column
conditional_weights = df['GPA'].apply(get_weight)

# multiply the conditional weights by the weights for the GPA column
weighted_gpa = df['GPA'] * conditional_weights * weights['GPA']

# multiply the other columns by their weights
weighted_others = df[['Test Scores', 'Attendance', 'Participation']] * pd.Series(weights)[['Test Scores', 'Attendance', 'Participation']]

# combine the weighted factors into a single score
total_weighted = pd.concat([weighted_gpa, weighted_others], axis=1)
normalized = (total_weighted - total_weighted.min()) / (total_weighted.max() - total_weighted.min())
df['Ranking Score'] = normalized.mean(axis=1) * 100